In [33]:
import pandas as pd
import numpy as np

# TODO: disentangle EE and ES!!!!
#  convert this matlab code to python list format
filenames = ['interleaved_incomplete',
             'block_incomplete',
             'block_complete',
             'block_complete_simple',
            ]

filenames2 = ['exp_1_interleaved_incomplete',
              'exp_2_block_incomplete',
              'exp_3_block_complete',
              'exp_4_block_complete_simple',
            ]


for f, f2 in zip(filenames, filenames2):
    df = pd.read_csv(f'data/behavior/csv/{f}.csv')

    df.drop(['Unnamed: 0',
             'prolific_id',
             'cont1', 'cont2',
             'dbtime', '10',
             '26', '15', '16',
             'dist', 'plot'], axis=1, inplace=True)

    df.rename(columns={'17': 'chose_right', 'elic': 'phase',
              'sub': 'sub_id', '25': 'reversed'}, inplace=True)
    df['chose_right'] = (df.chose_right > 0).astype(int)

    count = 0
    for sub in df.sub_id.unique():

        if np.isnan(sub):
            # drop nan sub
            df.drop(df[df.sub_id == sub].index, inplace=True)

        LE = (df.phase == -1) & (df.cond != -1) & (df.sub_id == sub)
        # count += sum(LE) == 120
        df.loc[LE, 'phase'] = 'LE'
        df.loc[LE, 'op1'] = 'E'
        df.loc[LE, 'op2'] = 'E'

        ES = (df.phase == 0) & (df.sub_id == sub) & (df.op2.isin(
            [0, -1])) & (df.op1.isin([1, -1, 0])) & (df.sub_id == sub)
        # assert len(LE[ES]) == 0
        # assert all(LE[ES] == False)
        # print(sum(ES))
        df.loc[ES, 'phase'] = 'ES'

        SP = (df.phase == 2) & (df.sub_id == sub)
        df.loc[SP, 'phase'] = 'SP'
        df.loc[SP & (df.catch == 0), 'op1'] = 'E'
        df.loc[SP & (df.catch == 1), 'op1'] = 'S'
        df.loc[SP, 'op2'] = np.NaN
        df.loc[SP, 'p2'] = np.NaN
        df.loc[SP, 'ev2'] = np.NaN
        df.loc[SP, 'cond'] = np.NaN

        df.loc[ES, 'op1'] = 'E'
        df.loc[ES, 'op2'] = 'S'
        df.loc[ES, 'cond'] = np.NaN

    df.loc[df.catch == -1, 'catch'] = 0
    df.loc[df.op2 == -1, 'op2'] = np.NaN
    df.loc[df.p2 == -1, 'op2'] = np.NaN
    df.rename(columns={'catch': 'catch_trial'}, inplace=True)
    df.to_csv(f'data/behavior/reformat/{f2}.csv', index_label='index')


filenames = [
             'block_complete_mixed',
             'block_complete_mixed_2s',
             'block_complete_mixed_2s_amb_heuristic',
             'block_complete_mixed_2s_amb_final',
             'incentivize']

filenames2 = [
              'exp_5_block_complete_mixed',
              'exp_6_block_complete_mixed_2s',
              'exp_7_block_complete_mixed_2s_deactivate',
              'exp_8_block_complete_mixed_2s_amb',
              'exp_9_incentives']


for f, f2 in zip(filenames, filenames2):
    df = pd.read_csv(f'data/behavior/csv/{f}.csv')

    df.drop(['Unnamed: 0',
             'prolific_id',
             'cont1', 'cont2',
             'dbtime', '10',
             '26', '15', '16',
             'dist', 'plot'], axis=1, inplace=True)

    df.rename(columns={'17': 'chose_right', 'elic': 'phase',
              'sub': 'sub_id', '25': 'reversed'}, inplace=True)
    df['chose_right'] = (df.chose_right > 0).astype(int)

    

    for sub in df.sub_id.unique():
        if np.isnan(sub):
            # drop nan sub
            df.drop(df[df.sub_id == sub].index, inplace=True)

        LE = (df.phase == -1) & (df.op1==1) & (df.op2==1) & (df.sub_id == sub)
        # count += sum(LE) == 120
        df.loc[LE, 'phase'] = 'LE'
        df.loc[LE, 'op1'] = 'E'
        df.loc[LE, 'op2'] = 'E'

        ES = (df.phase == 0) & (df.sub_id == sub) & (df.op2.isin(
            [0] )) & (df.op1.isin([0, 1])) & (df.sub_id == sub)
        # assert len(LE[ES]) == 0
        # assert all(LE[ES] == False)
        # print(sum(ES))
        df.loc[ES, 'phase'] = 'ES'

        SP = (df.phase == 2) & (df.sub_id == sub)
        df.loc[SP, 'phase'] = 'SP'
        df.loc[SP & (df.catch == 0), 'op1'] = 'E'
        df.loc[SP & (df.catch == 1), 'op1'] = 'S'
        df.loc[SP, 'op2'] = np.NaN
        df.loc[SP, 'p2'] = np.NaN
        df.loc[SP, 'ev2'] = np.NaN
        df.loc[SP, 'cond'] = np.NaN
        df.loc[SP, 'catch'] = 0

        df.loc[ES, 'op1'] = 'E'
        df.loc[ES, 'op2'] = 'S'
        df.loc[ES, 'cond'] = np.NaN

        EE = (df.phase == 0) & (df.op2 == 1) & (df.sub_id == sub)
        assert all((EE[ES] == False))
        df.loc[EE, 'phase'] = 'EE'
        df.loc[EE, 'op1'] = 'E'
        df.loc[EE, 'op2'] = 'E'
        df.loc[EE, 'cond'] = np.NaN
            # print(sum(EE)) 

        if 'amb' in f:
            EA = (df.phase == 0) & (df.op2 == 2) & (
                df.op1 == 1) & (df.sub_id == sub)
            df.loc[EA, 'phase'] = 'EA'

            df.loc[EA, 'op1'] = 'E'
            df.loc[EA, 'op2'] = 'A'
            df.loc[EA, 'cond'] = np.NaN

            SA = (df.phase == 0) & (df.op2 == 2) & (
                df.op1 == 0) & (df.sub_id == sub)

            df.loc[SA, 'phase'] = 'SA'
            df.loc[SA, 'op1'] = 'S'
            df.loc[SA, 'op2'] = 'A'
            df.loc[SA, 'cond'] = np.NaN

    df.loc[df.catch == -1, 'catch'] = 0
    df.loc[df.op2 == -1, 'op2'] = np.NaN
    df.loc[df.p2 == -1, 'op2'] = np.NaN
    df.rename(columns={'catch': 'catch_trial'}, inplace=True)
    df.to_csv(f'data/behavior/reformat/{f2}.csv', index_label='index')


In [43]:
import pandas as pd

df = pd.read_csv('./data/behavior/reformat/exp_2_block_incomplete.csv')
where = df.groupby('sub_id').phase.count().isin([288, 258])
ids = where.index[where.values]
df = df[df.sub_id.isin(ids)]

# remove training
df = df[df.sess >= 0]
# remove catch trials
df = df[df.catch_trial == 0]

df.groupby('sub_id').phase.value_counts()
# df = df[df.sub_id.isin(ids)]
# df = df[df.sess == 0]
# sum((df.phase=='ES') & (df.catch_trial == 1))
# df2 = df.groupby(['sub_id', 'phase', 'cond'], as_index=False)['corr'].mean()

# df2 = df2[df2.phase == 'LE']

# sns.set_palette('Set2')
# sns.barplot(data=df2, x='cond', y='corr', alpha=.5, zorder=-1)
# sns.stripplot(data=df2, x='cond', y='corr', palette='Set2',
            #   edgecolor='white', linewidth=1., size=8, zorder=0, alpha=.9)


sub_id        phase
1.402937e+08  LE       120
              ES        88
              SP         8
1.942133e+08  LE       120
              ES        88
                      ... 
9.937018e+09  ES        88
              SP         8
9.946139e+09  LE       120
              ES        88
              SP         8
Name: phase, Length: 291, dtype: int64

In [25]:
# check length
path =  './data/behavior/reformat2/exp_7_block_complete_mixed_2s_deactivate.csv'
df = pd.read_csv(path)
where = df.groupby('sub_id').phase.count().isin([648, 658])
ids = where.index[where.values]
df = df[df.sub_id.isin(ids)]
df = df[df.sess == 1]
print('Nsub=', len(df.sub_id.unique()))
print('phase=', df.phase.unique())
df.groupby(['sub_id', 'catch_trial']).phase.value_counts().to_frame()

Nsub= 90
phase= ['LE' 'EE' 'ES' 'EA' 'SA' 'SP']


phase
sub_id       catch_trial phase       
1.246278e+08 0.0         LE       120
                         ES        64
                         EE        56
                         EA        16
                         SA        16
...                               ...
9.864280e+09 0.0         EE        56
                         EA        16
                         SA        16
                         SP        10
             1.0         ES         8

[630 rows x 1 columns]

In [32]:
import pandas as pd
# check length
path =  './data/behavior/reformat2/exp_8_block_complete_mixed_2s_amb.csv'
df = pd.read_csv(path)
where = df.groupby('sub_id').phase.count().isin([648, 658])

ids = where.index[where.values]
df = df[df.sub_id.isin(ids)]
df = df[df.sess == 1]
print('Nsub=', len(df.sub_id.unique()))
print('phase=', df.phase.unique())
# ddd = 
df.groupby(['sub_id', 'catch_trial']).phase.value_counts().to_frame()


Nsub= 90
phase= ['LE' 'ES' 'EE' 'EA' 'SA' 'SP']


phase
sub_id       catch_trial phase       
6.287760e+06 0.0         LE       120
                         ES        64
                         EE        56
                         EA        16
                         SA        16
...                               ...
9.978490e+09 0.0         EE        56
                         EA        16
                         SA        16
                         SP        10
             1.0         ES         8

[630 rows x 1 columns]